# *Import Libraries*

In [2]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
import sys 
sys.path.append('/home/bhustali/.conda/envs/tf2/svcca-master')
import cca_core

# Simple Example

In [227]:
# # assume A_fake has 20 neurons and we have their activations on 2000 datapoints
# A_fake = np.random.randn(20, 2000)

# # B_fake has 50 neurons with activations on the same 1000 datapoints
# # Note A and B do *not* have to have the same number of neurons
# B_fake = np.random.randn(50, 1000)

# # computing CCA simliarty between A_fake, B_fake
# # We expect similarity should be very low, because the random activations are not correlated
# results = cca_core.get_cca_similarity(A_fake, B_fake,compute_dirns=True, verbose=True)

# print("Returned Information:  \n")
# print(results.keys())
# print("Single number for summarizing similarity")
# print('{:.4f}'.format(np.mean(results["cca_coef1"])))

adding eps to diagonal and taking inverse
taking square root
dot products...
trying to take final svd
computed everything!
Returned Information:  

dict_keys(['coef_x', 'invsqrt_xx', 'full_coef_x', 'full_invsqrt_xx', 'coef_y', 'invsqrt_yy', 'full_coef_y', 'full_invsqrt_yy', 'neuron_means1', 'neuron_means2', 'cca_coef1', 'cca_coef2', 'x_idxs', 'y_idxs', 'mean', 'sum', 'cca_dirns1', 'cca_dirns2'])
Single number for summarizing similarity
0.0841


# SVCCA Layers

In [1]:
for k in range(1,587):
    
    data_A = scipy.io.loadmat('/home/bhustali/data/mat/neuron_output (586).mat')
    data_B = scipy.io.loadmat('/home/bhustali/data/mat/neuron_output ' + '('  + str(k)+ ')' + '.mat')
    
    S = np.zeros((8,8))
    
    for i in range (1,9):
        for j in range (1,9):
            
            #extract layer outputs l_1 and l_2
            A_layer = data_A['layer_' + str(i)].transpose()
            B_layer = data_B['layer_' + str(j) ].transpose()
            
            # Mean subtract activations
            '''
            a = np.array([[1, 2], [3, 4]])
            np.mean(a, axis=1) = array([1.5, 3.5]) 
            '''
            c_A_layer = A_layer - np.mean(A_layer, axis=1, keepdims=True)
            c_B_layer = B_layer - np.mean(B_layer, axis=1, keepdims=True)
            
            #Singular Value Decomposition(SVD)
            #obtain l_1' and l_2'
            '''
            U, S, Vh = np.linalg.svd(A, full_matrices=False)

            U  = [mxs] = Left-singular vector of A
            S  = [sxs] = Singular values of A
            Vh = [sxp] = Right-singular vectors of A
            '''
            u_A, s_A, vh_A = np.linalg.svd(c_A_layer, full_matrices=False)
            u_B, s_B, vh_B = np.linalg.svd(c_B_layer, full_matrices=False)
            
            sv = 10 #select an appropriate value

#             print("Percentage of variance explained by 'sv' singular vectors", 100*np.sum(s_A[:sv])/np.sum(s_A))

            '''singular vectors = S * Vh 
            Equivalent to Uh * A = Uh* (U*S*Vh) = S*Vh
            '''
            #We compute the subspace with 'sv' largest singular values
            sv_A_layer = np.matmul(s_A[:sv]*np.eye(sv), vh_A[:sv])
            sv_B_layer = np.matmul(s_B[:sv]*np.eye(sv), vh_B[:sv])
           
            svcca_results = cca_core.get_cca_similarity(sv_A_layer, sv_B_layer, epsilon=0, threshold=1,compute_dirns=True, verbose=False)
        
            S[i-1,j-1] = np.mean(svcca_results["cca_coef1"])
            
            fig, ax = plt.subplots()
            ax.matshow(S, cmap=plt.cm.Blues)
            ax.set_xticklabels([0,'B1','B2','B3','B4','B5','B6','B7','B8'])
            ax.set_yticklabels([0,'A1','A2','A3','A4','A5','A6','A7','A8'])

            for l in range(8):
                for m in range(8):
                    c = S[m,l]
                    ax.text(l, m, f"{c:.2f}", va='center', ha='center')

            plt.savefig('/home/bhustali/data/movie/' + str(k) + '.png',dpi = 100)
            
            plt.close('all')

NameError: name 'scipy' is not defined

# How do layer outputs change during optimization?

Here, we measure the correlation of the output after each iteration (data_B) with the outputs of the layers before training (data_A)

In [5]:
# s = np.zeros((587,8))

# #outputs of the layers before training
# data_A = scipy.io.loadmat('data/mat/neuron_output (0).mat')

# for i in range (1,9):

#     for k in range(0,587):
        
#         #output after each iteration
#         data_B = scipy.io.loadmat('data/mat/neuron_output ' + '('  + str(k)+ ')' + '.mat')

#         j = i

#         A_layer = data_A['layer_' + str(i)].transpose()
#         B_layer = data_B['layer_' + str(j) ].transpose()

#         #shift mean to 0    
#         c_A_layer = A_layer - np.mean(A_layer, axis=1, keepdims=True)
#         c_B_layer = B_layer - np.mean(B_layer, axis=1, keepdims=True)
    
#         # Singular Value Decomposition(SVD)
#         u_A, s_A, vh_A = np.linalg.svd(c_A_layer, full_matrices=False)
#         u_B, s_B, vh_B = np.linalg.svd(c_B_layer, full_matrices=False)

#         sv = 10 #select an appropriate value

#         sv_A_layer = np.matmul(s_A[:sv]*np.eye(sv), vh_A[:sv])
#         sv_B_layer = np.matmul(s_B[:sv]*np.eye(sv), vh_B[:sv])

#         #compute similarity
#         svcca_results = cca_core.get_cca_similarity(sv_A_layer, sv_B_layer, epsilon=0, threshold=1,
#                                                     compute_dirns=True, verbose=False)

#         s[k,i-1] = np.mean(svcca_results["cca_coef1"])

# #Plotting 
# fig, ax = plt.subplots()
# for i in range (0,8):
#     ax.plot(s[:,i], label = str(i+1))
# ax.set(xlabel='iterations', ylabel='rho',
#        title='Convergence')
# ax.grid()
# plt.legend()

# # fig.savefig('divergence.png', dpi = 500)        